In [98]:
"""
This script can be used to evaluate a trained model on 3D pose/shape and masks/part segmentation. You first need to download the datasets and preprocess them.
Example usage:
```
python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20
```
Running the above command will compute the MPJPE and Reconstruction Error on the Human3.6M dataset (Protocol I). The ```--dataset``` option can take different values based on the type of evaluation you want to perform:
1. Human3.6M Protocol 1 ```--dataset=h36m-p1```
2. Human3.6M Protocol 2 ```--dataset=h36m-p2```
3. 3DPW ```--dataset=3dpw```
4. LSP ```--dataset=lsp```
5. MPI-INF-3DHP ```--dataset=mpi-inf-3dhp```
"""

import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2
import os
import argparse
import json
from collections import namedtuple
from tqdm import tqdm
import torchgeometry as tgm

import config
import constants
from models import hmr, SMPL
from datasets import BaseDataset
from utils.imutils import uncrop
from utils.pose_utils import reconstruction_error
from utils.part_utils import PartRenderer

from scipy.spatial.transform import Rotation as R

# Define command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', default=None, help='Path to network checkpoint')
parser.add_argument('--dataset', default='h36m-p1', choices=['h36m-p1', 'h36m-p2', 'lsp', '3dpw', 'mpi-inf-3dhp'], help='Choose evaluation dataset')
parser.add_argument('--log_freq', default=50, type=int, help='Frequency of printing intermediate results')
parser.add_argument('--batch_size', default=32, help='Batch size for testing')
parser.add_argument('--shuffle', default=False, action='store_true', help='Shuffle data')
parser.add_argument('--num_workers', default=8, type=int, help='Number of processes for data loading')
parser.add_argument('--result_file', default=None, help='If set, save detections to a .npz file')

def run_evaluation(model, dataset_name, dataset, result_file,
                   batch_size=32, img_res=224, 
                   num_workers=32, shuffle=False, log_freq=50):
    """Run evaluation on the datasets and metrics we report in the paper. """

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Transfer model to the GPU
    model.to(device)

    # Load SMPL model
    smpl_neutral = SMPL(config.SMPL_MODEL_DIR,
                        create_transl=False).to(device)
    smpl_male = SMPL(config.SMPL_MODEL_DIR,
                     gender='male',
                     create_transl=False).to(device)
    smpl_female = SMPL(config.SMPL_MODEL_DIR,
                       gender='female',
                       create_transl=False).to(device)
    
    renderer = PartRenderer()
    
    # Regressor for H36m joints
    J_regressor = torch.from_numpy(np.load(config.JOINT_REGRESSOR_H36M)).float()
    
    save_results = result_file is not None
    # Disable shuffling if you want to save the results
    if save_results:
        shuffle=False
    # Create dataloader for the dataset
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    
    # Pose metrics
    # MPJPE and Reconstruction error for the non-parametric and parametric shapes
    mpjpe = np.zeros(len(dataset))
    recon_err = np.zeros(len(dataset))
    mpjpe_smpl = np.zeros(len(dataset))
    recon_err_smpl = np.zeros(len(dataset))
    
    # Including mean orientation error
    moe = np.zeros(len(dataset))

    # Shape metrics
    # Mean per-vertex error
    shape_err = np.zeros(len(dataset))
    shape_err_smpl = np.zeros(len(dataset))

    # Mask and part metrics
    # Accuracy
    accuracy = 0.
    parts_accuracy = 0.
    # True positive, false positive and false negative
    tp = np.zeros((2,1))
    fp = np.zeros((2,1))
    fn = np.zeros((2,1))
    parts_tp = np.zeros((7,1))
    parts_fp = np.zeros((7,1))
    parts_fn = np.zeros((7,1))
    # Pixel count accumulators
    pixel_count = 0
    parts_pixel_count = 0

    # Store SMPL parameters
    smpl_pose = np.zeros((len(dataset), 72))
    smpl_betas = np.zeros((len(dataset), 10))
    smpl_camera = np.zeros((len(dataset), 3))
    pred_joints = np.zeros((len(dataset), 17, 3))

    eval_pose = False
    eval_masks = False
    eval_parts = False
    eval_orientation = True # Adding the orientation parameter
    # Choose appropriate evaluation for each dataset
    if dataset_name == 'h36m-p1' or dataset_name == 'h36m-p2' or dataset_name == '3dpw' or dataset_name == 'mpi-inf-3dhp':
        eval_pose = True
    elif dataset_name == 'lsp':
        eval_masks = True
        eval_parts = True
        annot_path = config.DATASET_FOLDERS['upi-s1h']
    #if dataset_name == '3dpw':
    #    eval_orientation == True

    joint_mapper_h36m = constants.H36M_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.H36M_TO_J14
    joint_mapper_gt = constants.J24_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.J24_TO_J14
    # Iterate over the entire dataset
    for step, batch in enumerate(tqdm(data_loader, desc='Eval', total=len(data_loader))):
        # Get ground truth annotations from the batch
        gt_pose = batch['pose'].to(device)
        gt_betas = batch['betas'].to(device)
        gt_vertices = smpl_neutral(betas=gt_betas, body_pose=gt_pose[:, 3:], global_orient=gt_pose[:, :3]).vertices
        images = batch['img'].to(device)
        gender = batch['gender'].to(device)
        curr_batch_size = images.shape[0]
        
        with torch.no_grad():
            pred_rotmat, pred_betas, pred_camera = model(images)
            pred_output = smpl_neutral(betas=pred_betas, body_pose=pred_rotmat[:,1:], global_orient=pred_rotmat[:,0].unsqueeze(1), pose2rot=False)
            pred_vertices = pred_output.vertices

        if save_results:
            rot_pad = torch.tensor([0,0,1], dtype=torch.float32, device=device).view(1,3,1)
            rotmat = torch.cat((pred_rotmat.view(-1, 3, 3), rot_pad.expand(curr_batch_size * 24, -1, -1)), dim=-1)
            pred_pose = tgm.rotation_matrix_to_angle_axis(rotmat).contiguous().view(-1, 72)
            smpl_pose[step * batch_size:step * batch_size + curr_batch_size, :] = pred_pose.cpu().numpy()
            smpl_betas[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_betas.cpu().numpy()
            smpl_camera[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_camera.cpu().numpy()
        
        # Orientation evaluation
        if eval_orientation:
            # Get ground truth orientation (already stored in gt_pose)
            
            #print('rotmat',pred_rotmat)
            #print('rotmat size',pred_rotmat.size())
            
            # Absolute error (MPJPE)
            #print(gt_pose.size())
            #gt_orientation = gt_pose.cpu().numpy()
            #print(np.size(gt_orientation))
            #gt_orientation_reshaped = np.reshape(gt_orientation, (32, -1)) # reshaping the sequence file to make it inputtable on R.from_rotvec
            #print(np.size(gt_orientation_reshaped))
            #r = R.from_rotvec(gt_orientation_reshaped) # Have to find a way to input the 32 axis-angle vectors here (unfolding them somehow)
            #gt_orientation_matrix = r.as_dcm()
            #gt_orientation_tensor = torch.as_tensor(seq_matrix,dtype=torch.float, device='cuda').unsqueeze(0)
            
            gt_orientation_reshaped = torch.zeros(32,24,3) # Have to reshape from 32,72 to 32,24,3 to input into the rotation object
            i = 0
            for row in gt_pose:
                gt_orientation_reshaped[i] = torch.reshape(row,(24, -1))
                i+=1
            gt_orientation_as_matrix = np.zeros((32, 24, 3, 3)) # Using numpy here because it works with the rotation library
            i = 0
            for row in gt_orientation_reshaped:
                r = R.from_rotvec(row)
                gt_orientation_as_matrix[i] = r.as_dcm()
                i+=1

            orientation_error = pred_rotmat - torch.as_tensor(gt_orientation_as_matrix, dtype=torch.float, device='cuda')
            moe[step * batch_size:step * batch_size + curr_batch_size] = orientation_error.mean(-1).mean(-1).mean(-1).cpu().numpy()
        
        # 3D pose evaluation
        if eval_pose:
            # Regressor broadcasting
            J_regressor_batch = J_regressor[None, :].expand(pred_vertices.shape[0], -1, -1).to(device)
            # Get 14 ground truth joints
            if 'h36m' in dataset_name or 'mpi-inf' in dataset_name:
                gt_keypoints_3d = batch['pose_3d'].cuda()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_gt, :-1]
            # For 3DPW get the 14 common joints from the rendered shape
            else:
                gt_vertices = smpl_male(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices_female = smpl_female(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices[gender==1, :, :] = gt_vertices_female[gender==1, :, :]
                gt_keypoints_3d = torch.matmul(J_regressor_batch, gt_vertices)
                gt_pelvis = gt_keypoints_3d[:, [0],:].clone()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_h36m, :]
                gt_keypoints_3d = gt_keypoints_3d - gt_pelvis 


            # Get 14 predicted joints from the mesh
            pred_keypoints_3d = torch.matmul(J_regressor_batch, pred_vertices)
            if save_results:
                pred_joints[step * batch_size:step * batch_size + curr_batch_size, :, :]  = pred_keypoints_3d.cpu().numpy()
            pred_pelvis = pred_keypoints_3d[:, [0],:].clone()
            pred_keypoints_3d = pred_keypoints_3d[:, joint_mapper_h36m, :]
            pred_keypoints_3d = pred_keypoints_3d - pred_pelvis 

            # Absolute error (MPJPE)
            error = torch.sqrt(((pred_keypoints_3d - gt_keypoints_3d) ** 2).sum(dim=-1)).mean(dim=-1).cpu().numpy()
            mpjpe[step * batch_size:step * batch_size + curr_batch_size] = error

            # Reconstuction_error
            r_error = reconstruction_error(pred_keypoints_3d.cpu().numpy(), gt_keypoints_3d.cpu().numpy(), reduction=None)
            recon_err[step * batch_size:step * batch_size + curr_batch_size] = r_error


        # If mask or part evaluation, render the mask and part images
        if eval_masks or eval_parts:
            mask, parts = renderer(pred_vertices, pred_camera)

        # Mask evaluation (for LSP)
        if eval_masks:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            # Dimensions of original image
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                # After rendering, convert imate back to original resolution
                pred_mask = uncrop(mask[i].cpu().numpy(), center[i], scale[i], orig_shape[i]) > 0
                # Load gt mask
                gt_mask = cv2.imread(os.path.join(annot_path, batch['maskname'][i]), 0) > 0
                # Evaluation consistent with the original UP-3D code
                accuracy += (gt_mask == pred_mask).sum()
                pixel_count += np.prod(np.array(gt_mask.shape))
                for c in range(2):
                    cgt = gt_mask == c
                    cpred = pred_mask == c
                    tp[c] += (cgt & cpred).sum()
                    fp[c] +=  (~cgt & cpred).sum()
                    fn[c] +=  (cgt & ~cpred).sum()
                f1 = 2 * tp / (2 * tp + fp + fn)

        # Part evaluation (for LSP)
        if eval_parts:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                pred_parts = uncrop(parts[i].cpu().numpy().astype(np.uint8), center[i], scale[i], orig_shape[i])
                # Load gt part segmentation
                gt_parts = cv2.imread(os.path.join(annot_path, batch['partname'][i]), 0)
                # Evaluation consistent with the original UP-3D code
                # 6 parts + background
                for c in range(7):
                   cgt = gt_parts == c
                   cpred = pred_parts == c
                   cpred[gt_parts == 255] = 0
                   parts_tp[c] += (cgt & cpred).sum()
                   parts_fp[c] +=  (~cgt & cpred).sum()
                   parts_fn[c] +=  (cgt & ~cpred).sum()
                gt_parts[gt_parts == 255] = 0
                pred_parts[pred_parts == 255] = 0
                parts_f1 = 2 * parts_tp / (2 * parts_tp + parts_fp + parts_fn)
                parts_accuracy += (gt_parts == pred_parts).sum()
                parts_pixel_count += np.prod(np.array(gt_parts.shape))

        # Print intermediate results during evaluation
        if step % log_freq == log_freq - 1:
            if eval_pose:
                print('MPJPE: ' + str(1000 * mpjpe[:step * batch_size].mean()))
                print('Reconstruction Error: ' + str(1000 * recon_err[:step * batch_size].mean()))
                print()
            if eval_masks:
                print('Accuracy: ', accuracy / pixel_count)
                print('F1: ', f1.mean())
                print()
            if eval_parts:
                print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
                print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
                print()
            if eval_orientation:
                print('Orientation error: ' + str(1000 * moe[:step * batch_size].mean()))

    # Save reconstructions to a file for further processing
    if save_results:
        np.savez(result_file, pred_joints=pred_joints, pose=smpl_pose, betas=smpl_betas, camera=smpl_camera)
    # Print final results during evaluation
    print('*** Final Results ***')
    print()
    if eval_pose:
        print('MPJPE: ' + str(1000 * mpjpe.mean()))
        print('Reconstruction Error: ' + str(1000 * recon_err.mean()))
        print()
    if eval_masks:
        print('Accuracy: ', accuracy / pixel_count)
        print('F1: ', f1.mean())
        print()
    if eval_parts:
        print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
        print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
        print()

In [99]:
if __name__ == '__main__':
    
    # python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20 // example code
    args = parser.parse_args(['--checkpoint=data/model_checkpoint.pt','--dataset=3dpw', '--log_freq=20', '--num_workers=11'])
    # Here we inserted our own arguments list
    
    model = hmr(config.SMPL_MEAN_PARAMS)
    checkpoint = torch.load(args.checkpoint)
    model.load_state_dict(checkpoint['model'], strict=False)
    model.eval()

    # Setup evaluation dataset
    dataset = BaseDataset(None, args.dataset, is_train=False)
    # Run evaluation
    run_evaluation(model, args.dataset, dataset, args.result_file,
                   batch_size=args.batch_size,
                   shuffle=args.shuffle,
                   log_freq=args.log_freq)

Eval:   2%|▏         | 20/1110 [00:54<34:48,  1.92s/it]  

MPJPE: 81.37874403282215
Reconstruction Error: 41.73969471900675

Orientation error: 0.6353915284657768


Eval:   4%|▎         | 40/1110 [01:04<13:42,  1.30it/s]

MPJPE: 93.01356022031261
Reconstruction Error: 45.72069009867473

Orientation error: 3.1753492330236757


Eval:   5%|▌         | 60/1110 [01:27<06:55,  2.53it/s]  

...

MPJPE: 99.71019655986686
Reconstruction Error: 58.42986403965147

Orientation error: 0.5629081468022582


Eval:  32%|███▏      | 360/1110 [04:54<05:58,  2.09it/s]

MPJPE: 99.22611367989923
Reconstruction Error: 58.74572166184633

Orientation error: 0.5314700730187952


Eval:  33%|███▎      | 363/1110 [04:54<04:30,  2.76it/s]


RuntimeError: DataLoader worker (pid 2993) is killed by signal: Killed. 

In [78]:
import torch
import numpy as np

torch.set_printoptions(profile="full")
np.set_printoptions(threshold=np.inf)

gt_pose = torch.rand(32,72)
a

#np.apply_along_axis(np.reshape( ,(24, -1)), -1, a)

gt_orientation_reshaped = torch.zeros(32,24,3) # Have to reshape from 32,72 to 32,24,3 to input into the rotation object
i = 0

for row in gt_pose:
    gt_orientation_reshaped[i] = torch.reshape(row,(24, -1))
    i+=1
    
gt_orientation_as_matrix = np.zeros((32, 24, 3, 3)) # Using numpy here because it works with the rotation library
i = 0

for row in gt_orientation_reshaped:
    r = R.from_rotvec(row)
    gt_orientation_as_matrix[i] = r.as_dcm()
    i+=1

In [82]:
#moe = np.zeros(32)
pred_rotmat = torch.rand(32,24,3,3)

orientation_error = pred_rotmat - torch.as_tensor(gt_orientation_as_matrix, dtype=torch.float, device='cpu')
#moe = orientation_error

In [97]:
orientation_error.mean(-1).mean(-1).mean(-1).cpu().numpy()

array([0.16691504, 0.21696417, 0.1693462 , 0.1735936 , 0.19022979,
       0.14186798, 0.1794252 , 0.1365677 , 0.2056535 , 0.21439867,
       0.20660938, 0.1984337 , 0.20832618, 0.18885958, 0.16714345,
       0.16724224, 0.19226904, 0.16536777, 0.21545953, 0.19916196,
       0.19647403, 0.17933144, 0.201521  , 0.21505421, 0.18387286,
       0.19260901, 0.17172949, 0.19314033, 0.16249119, 0.20120196,
       0.20672823, 0.16412131], dtype=float32)

In [62]:
b[0].size()

torch.Size([24, 3])

In [63]:
np.shape(r_matrix)

(32, 24, 3, 3)

In [67]:
r_matrix

array([[[[ 9.26009725e-01, -3.50589280e-01,  1.39975521e-01],
         [ 3.56104596e-01,  6.88200250e-01, -6.32115442e-01],
         [ 1.25281709e-01,  6.35190973e-01,  7.62126578e-01]],

        [[ 7.35272086e-01, -1.68586314e-01,  6.56470574e-01],
         [ 4.29591575e-01,  8.65084095e-01, -2.58999202e-01],
         [-5.24238531e-01,  4.72449112e-01,  7.08495447e-01]],

        [[ 8.54233739e-01,  6.62481768e-02,  5.15650946e-01],
         [ 3.90574167e-01,  5.72843757e-01, -7.20626013e-01],
         [-3.43127585e-01,  8.16982992e-01,  4.63467638e-01]],

        [[ 9.62842287e-01,  4.53150808e-03,  2.70026286e-01],
         [ 6.43964851e-02,  9.67166020e-01, -2.45851545e-01],
         [-2.62274326e-01,  2.54105008e-01,  9.30936530e-01]],

        [[ 8.69415079e-01, -4.21310535e-01,  2.58098535e-01],
         [ 4.93550682e-01,  7.16335294e-01, -4.93225578e-01],
         [ 2.29160422e-02,  5.56202462e-01,  8.30730808e-01]],

        [[ 7.59405096e-01, -6.03562522e-01,  2.42932466e-01]

In [65]:
a.size()

torch.Size([32, 72])

In [66]:
b.size()

torch.Size([32, 24, 3])

In [30]:
a

tensor([[0.0212, 0.0156, 0.4606, 0.4203, 0.0510, 0.0300, 0.4997, 0.1943, 0.6864,
         0.9201, 0.7787, 0.6476, 0.5857, 0.1304, 0.6822, 0.2462, 0.1651, 0.0784,
         0.8257, 0.6226, 0.0112, 0.4544, 0.4077, 0.3481, 0.3890, 0.7719, 0.7765,
         0.6140, 0.7380, 0.4730, 0.6231, 0.2299, 0.6805, 0.5320, 0.7710, 0.7926,
         0.9604, 0.0139, 0.4713, 0.9143, 0.4831, 0.2685, 0.1396, 0.5329, 0.6804,
         0.7584, 0.9821, 0.7646, 0.3415, 0.9464, 0.2441, 0.0794, 0.4197, 0.7124,
         0.8824, 0.9064, 0.9212, 0.4823, 0.7880, 0.2179, 0.5871, 0.2116, 0.0346,
         0.2590, 0.3778, 0.5263, 0.5251, 0.3056, 0.1014, 0.4634, 0.1300, 0.0323],
        [0.0578, 0.1821, 0.7800, 0.2085, 0.4984, 0.0994, 0.8356, 0.0217, 0.3303,
         0.4672, 0.3975, 0.7023, 0.4084, 0.9299, 0.3200, 0.0204, 0.0089, 0.2419,
         0.4409, 0.1575, 0.2967, 0.1805, 0.6218, 0.1428, 0.1057, 0.8027, 0.0794,
         0.4797, 0.8252, 0.5008, 0.1923, 0.3199, 0.9381, 0.8057, 0.2086, 0.2399,
         0.3309, 0.9643, 0.

In [31]:
b

tensor([[[0.0212, 0.0156, 0.4606],
         [0.4203, 0.0510, 0.0300],
         [0.4997, 0.1943, 0.6864],
         [0.9201, 0.7787, 0.6476],
         [0.5857, 0.1304, 0.6822],
         [0.2462, 0.1651, 0.0784],
         [0.8257, 0.6226, 0.0112],
         [0.4544, 0.4077, 0.3481],
         [0.3890, 0.7719, 0.7765],
         [0.6140, 0.7380, 0.4730],
         [0.6231, 0.2299, 0.6805],
         [0.5320, 0.7710, 0.7926],
         [0.9604, 0.0139, 0.4713],
         [0.9143, 0.4831, 0.2685],
         [0.1396, 0.5329, 0.6804],
         [0.7584, 0.9821, 0.7646],
         [0.3415, 0.9464, 0.2441],
         [0.0794, 0.4197, 0.7124],
         [0.8824, 0.9064, 0.9212],
         [0.4823, 0.7880, 0.2179],
         [0.5871, 0.2116, 0.0346],
         [0.2590, 0.3778, 0.5263],
         [0.5251, 0.3056, 0.1014],
         [0.4634, 0.1300, 0.0323]],

        [[0.0578, 0.1821, 0.7800],
         [0.2085, 0.4984, 0.0994],
         [0.8356, 0.0217, 0.3303],
         [0.4672, 0.3975, 0.7023],
         [0.4084, 